In [1]:
import os
for env in ["MKL_NUM_THREADS", "NUMEXPR_NUM_THREADS", "OMP_NUM_THREADS"]:
    os.environ[env] = "1"

import numpy as np
from numpy.random import default_rng
import numpy.random as rnd
from ttictoc import tic, toc

import approxbayescomp as abc

In [2]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)

tic()

ABC version: 0.1.0
Python version: 3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 15:50:57) 
[Clang 11.0.1 ]
Numpy version: 1.20.2


In [3]:
numIters = 10
popSize = 1000
smcArgs = {"verbose": True, "numProcs": 8}

In [4]:
rg = default_rng(123)

sample_sizes = [50, 250]
T = sample_sizes[-1]

# Frequency-Loss Model
sigma = 0.2
w1 = 15
w2 = 5
m1 = 10
m2 = 40
θ_True = sigma, w1, w2, m1, m2

freq = "bivariate poisson"
sev = "exponential"

# Aggregation process
psi = abc.Psi("sum")

claimsData = abc.simulate_claim_data(rg, T, freq, sev, θ_True)

xData1 = abc.compute_psi(claimsData[0][0], claimsData[0][1], psi)
xData2 = abc.compute_psi(claimsData[1][0], claimsData[1][1], psi)

xData = (xData1, xData2)

In [5]:
prior = abc.IndependentUniformPrior([(0, 2), (0, 50), (0, 50), (0, 100), (0, 100)])
model = abc.Model("bivariate poisson", "exponential", psi, prior)

In [6]:
params = ("sigma", "w1", "w2", "m1", "m2")

for ss in sample_sizes:
    xDataSS = np.vstack([xData1[:ss], xData2[:ss]]).T

    %time fit =  abc.smc(numIters, popSize, xDataSS, model, sumstats=abc.wass_2Ddist_ss, distance=abc.wass_2Ddist, **smcArgs)

Final population dists <= 43.39, ESS = 416
CPU times: user 2.5 s, sys: 235 ms, total: 2.74 s
Wall time: 18.7 s


Final population dists <= 43.81, ESS = 756
CPU times: user 4 s, sys: 383 ms, total: 4.38 s
Wall time: 1min 41s


In [7]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 121 secs = 2.01 mins
